In [ ]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import sqlite3
import sys
sys.path.append('../tools') # agregar la ruta de la carpeta tools
from tool import sqlite_tool,consulta_filtrada,handle_tool_call

In [4]:
# Inicialización

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key sin configurar")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


## Prompt de systema

In [5]:
system_message = "Eres un asistente útil para una inmobiliaria de Tunja Boyaca llamada MetroCuadrado. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

## Open Ai

In [6]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=sqlite_tool)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response= handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comenta o elimina la siguiente línea si prefiere omitir el audio por ahora.
    #talker(reply)
    
    return history

In [8]:
# Código Gradio más complejo, ya que no estamos usando la interfaz de chat predeterminada.
# Si pasamos inbrowser=True en la última línea, aparecerá inmediatamente una ventana Gradio.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image('../image/tunja.png',height=500,interactive=False)
    with gr.Row():
        entry = gr.Textbox(label="Agente metro_cuadrado:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True,)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


{'tabla': 'property', 'columnas': '*', 'condiciones': "estado='venta_apartamento' AND barrio_comun='la esperanza'"}


In [ ]:
with gr.Blocks() as demo:
    with gr.Group():
        with gr.Column(scale=1):
            imagen = gr.Image('../image/tunja.png',width=250,interactive=False)
        with gr.Column(scale=2):
            btn1 = gr.Button("Button 1")
            btn2 = gr.Button("Button 2")
demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [43]:
# Función para manejar los mensajes del chat
def do_entry(message, history):
    history += [{"role": "user", "content": message}]
    # Aquí puedes añadir la lógica para generar respuestas del chatbot
    response = f"Respuesta al mensaje: {message}"  # Ejemplo simple
    history += [{"role": "assistant", "content": response}]
    return "", history

# Creación de la interfaz
with gr.Blocks() as ui:
    # Primera fila: Chatbot e imagen fija lado a lado
    with gr.Row():
        # Columna para el chatbot
        with gr.Column():
            chatbot = gr.Chatbot(height=500, type="messages")
            # Caja de texto para entrada
            entry = gr.Textbox(label="Chatea con nuestro Agente de IA:")
            # Botón de limpiar
            clear = gr.Button("Clear")
        
        # Columna para la imagen fija
        with gr.Column():
            # Puedes reemplazar "path_to_your_image.jpg" con la ruta a tu imagen
            image = gr.Image(value="../image/tunja.png", height=250,width=250,interactive=False)

    # Configuración de eventos
    entry.submit(
        do_entry,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

# Lanzar la interfaz
ui.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [35]:
import gradio as gr

def image_classifier(inp):
    return {'cat': 0.3, 'dog': 0.7}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


In [36]:
import gradio as gr

hello_world = gr.Interface(lambda name: "Hello " + name, "text", "text")
bye_world = gr.Interface(lambda name: "Bye " + name, "text", "text")

demo = gr.TabbedInterface([hello_world, bye_world], ["Hello World", "Bye World"])

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


In [37]:
import gradio as gr
def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    gr.Markdown("Start typing below and then click **Run** to see the output.")
    with gr.Row():
        inp = gr.Textbox(placeholder="What is your name?")
        out = gr.Textbox()
    btn = gr.Button("Run")
    btn.click(fn=update, inputs=inp, outputs=out)

demo.launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


In [38]:
import gradio as gr

def welcome(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Hello World!
    Start typing below to see the output.
    """)
    inp = gr.Textbox(placeholder="What is your name?")
    out = gr.Textbox()
    inp.change(welcome, inp, out)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [4]:
from typing import Annotated, Literal

from annotated_types import Gt

from pydantic import BaseModel


class Fruit(BaseModel):
    name: str = "vacio"
    color: Literal['red', 'green']  
    weight: Annotated[float, Gt(0)]  
    bazam: dict[str, list[tuple[int, bool, float]]]  


print(
    Fruit(
        color='red',
        weight=4.2,
        bazam={'foobar': [(1, True, 0.1)]},
    )
)
#> name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}

name='vacio' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}
